In [1]:
from utils import server

llm = server()
data = llm.greet(patient_details="{'name': 'Avijit bhuin', 'gender': 'male', 'age': '22', 'height': '180', 'weight': '68', 'medical_issue': 'belly ache and headache.'}")

"Hello Avijit, welcome to our medical facility. I'm Doctor AI, and I'll be assisting you with your diagnosis today. I see that you're experiencing a belly ache and headache. Before we proceed, I'd like to ask, do you have any pre-existing medical conditions, such as allergies, chronic illnesses, or previous surgeries, that we should be aware of while evaluating your current symptoms? This information will help us provide you with the most accurate and effective care possible."


In [2]:
print(data)

"Hello Avijit, welcome to our medical facility. I'm Doctor AI, and I'll be assisting you with your diagnosis today. I see that you're experiencing a belly ache and headache. Before we proceed, I'd like to ask, do you have any pre-existing medical conditions, such as allergies, chronic illnesses, or previous surgeries, that we should be aware of while evaluating your current symptoms? This information will help us provide you with the most accurate and effective care possible."


In [11]:
from utils import Search_Tool

search = Search_Tool()

urls = search.search('medication for typhoid', 5)
urls


[{'title': 'Typhoid Fever: Causes, Symptoms & Treatment',
  'description': 'Featured snippet from the webमराठी मध्येथांबवाIn EnglishPauseYour healthcare provider will treat typhoid fever with antibiotics, which may include:Ciprofloxacin, levoflaxin or ofloxacin.Ceftriaxone, cefotaxime or cefixime.Azithromycin.Typhoid Fever: Causes, Symptoms & TreatmentCleveland Clinichttps://my.clevelandclinic.org› health › diseases › 177...Cleveland Clinichttps://my.clevelandclinic.org› health › diseases › 177...About featured snippets•Feedback',
  'url': 'https://my.clevelandclinic.org/health/diseases/17730-typhoid-fever#:~:text=Your%20healthcare%20provider%20will%20treat,Azithromycin.'},
 {'title': 'Typhoid fever - Diagnosis & treatment',
  'description': '28 Jan 2023—Antibiotic therapy is the only effective treatment for typhoid fever. Commonly prescribed antibiotics. The medicine you get totreat typhoid...',
  'url': 'https://www.mayoclinic.org/diseases-conditions/typhoid-fever/diagnosis-treatment

In [12]:
urls[1]

{'title': 'Typhoid fever - Diagnosis & treatment',
 'description': '28 Jan 2023—Antibiotic therapy is the only effective treatment for typhoid fever. Commonly prescribed antibiotics. The medicine you get totreat typhoid...',
 'url': 'https://www.mayoclinic.org/diseases-conditions/typhoid-fever/diagnosis-treatment/drc-20378665'}

In [13]:
import json
from bs4 import BeautifulSoup
import requests

html = requests.get(urls[1]['url']).text

soup = BeautifulSoup(html, 'html.parser')

data = []
for h2 in soup.find_all(['h2', 'h3', 'h4']):
    section = {}
    section['h'] = h2.text.strip()
    p = h2.find_next(['p', 'ul'])
    section['p'] = ''
    section['ul'] = ''
    while p and p.name in ['p', 'ul']:
        if p.name == 'p':
            section['p'] += p.text.strip() + ' '
        elif p.name == 'ul':
            for li in p.find_all('li'):
                section['ul'] += li.text.strip() + ', '
        p = p.find_next(['p', 'ul'])
    section['p'] = section['p'].strip()
    section['ul'] = section['ul'].strip(', ').strip()

    if "medi" in section['h'].lower() or "medi" in section['h'].lower() or "treat" in section['h'].lower() or "prescr" in section['h'].lower() or "antibi" in section['h'].lower() or "diagnos" in section['h'].lower():
        data.append(section)

data

[{'h': 'Diagnosis',
  'p': "Your health care provider may suspect typhoid fever based on your symptoms, and your medical and travel history. The diagnosis is often confirmed by growing the Salmonella enterica serotype typhi in a sample of your body fluid or tissue. A sample of your blood, stool, urine or bone marrow is used. The sample is placed in an environment where bacteria grow easily. The growth, called a culture, is checked under a microscope for the typhoid bacteria. A bone marrow culture often is the most sensitive test for Salmonella typhi. A culture test is the most common diagnostic test. But other testing may be used to confirm typhoid fever. One is a test to detect antibodies to typhoid bacteria in your blood. Another test checks for typhoid DNA in your blood. Antibiotic therapy is the only effective treatment for typhoid fever. The medicine you get to treat typhoid fever may depend on where you picked up the bacteria. Strains picked up in different places respond better 

In [16]:
prompt = str(data) + '''Based on this data Construct the answer in this JSON format.
                        {"disease":str, "meds":list(str), "other_treatments": list(str)}
                        Question:
                        what are the medications and treatments for [[[disease]]]?
                        Answer:'''

In [1]:
from utils import server
bot = server()

results = bot.generate_conclusion(disease="Typhoid")

In [ ]:
import asyncio
import time
from typing import List, Any
from prompts import *
import nest_asyncio
from utils import LLM
nest_asyncio.apply()

class RateLimitedAPI:
    def __init__(self):
        self.rate_limit_delay = 3  # 1 second delay
        self.semaphore = asyncio.Semaphore(1)  # Control concurrent requests
        self.last_request_time = 0.0
        self.llm_client = LLM()

    async def ask_mistral_with_rate_limit(self, prompt: str) -> Any:
        async with self.semaphore:
            # Calculate time to wait based on last request
            current_time = time.time()
            time_since_last_request = current_time - self.last_request_time
            if time_since_last_request < self.rate_limit_delay:
                await asyncio.sleep(self.rate_limit_delay - time_since_last_request)
            
            # Make the API call
            try:
                result = await self.llm_client.ask_Mistral(
                    question=prompt,
                    model="open-mistral-nemo",
                    JSON=True
                )
                self.last_request_time = time.time()
                return result
            except Exception as e:
                print(f"Error processing prompt: {e}")
                return None

    async def process_prompts(self, results: List[Any], disease: str) -> List[Any]:
        prompts = [
            str(i) + WEBPAGE_EXTRACTOR.replace("[[[disease]]]", disease)
            for i in results
        ]
        
        # Create tasks for all prompts
        tasks = [
            self.ask_mistral_with_rate_limit(prompt)
            for prompt in prompts
        ]
        
        # Execute all tasks concurrently
        results = await asyncio.gather(*tasks)
        return results

    def process_prompts_sync(self, results: List[Any], disease: str) -> List[Any]:
        """Synchronous wrapper for the async function"""
        return asyncio.run(self.process_prompts(results, disease))

# Usage example:

api_client = RateLimitedAPI()
results = api_client.process_prompts_sync(results, "Typhoid")